In [ ]:
!pip install -r requirements.txt

In [1]:
from model.encode_decode_model import *
from model.text_model import *
from utils.config import *
from scripts.image_text_gen import *
from scripts.train_encoder import *

if __name__ == '__main__':
    
    encoder_checkpoint = 'checkpoint\\encoder.pt'
    decoder_checkpoint = 'checkpoint\\decoder.pt'
    EncoderDecoder_checkpoint = 'checkpoint\\text_gen_train_for_100_iters.pt'
    
    # load encoder
    if encoder_checkpoint == None:    
        # training encoder
        encoder = Encoder(config).to(config.device)
        train_encoder(encoder)
    else : 
        encoder = Encoder(config).to(config.device)
        load_checkpoint(encoder, encoder_checkpoint)
    print('=======================================')
    print('-       load encoder complete         -')
    print('=======================================')
    # load decoder
    if decoder_checkpoint == None:
        # training decoder
        decoder = Decoder(config).to(config.device)
        optim = torch.optim.AdamW(decoder.parameters(), lr=config.lr)
        train_gen(decoder, optim)
    
    else:
        decoder = Decoder(config).to(config.device)
        load_checkpoint(decoder, decoder_checkpoint)
    print('=======================================')
    print('-       load decoder complete         -')
    print('=======================================')
    # load EncoderDecoder
    if EncoderDecoder_checkpoint == None:
        model = EncoderDecoderModel(encoder=encoder, decoder=decoder, config=config).to(config.device)
        optim = torch.optim.AdamW(model.parameters(), lr=config.lr)
        train_image_text(model, optim, config.train_df, config.valid_df, config.train_image_dir, config.valid_image_dir)
    else :
        model = EncoderDecoderModel(encoder=encoder, decoder=decoder, config=config).to(config.device)
        load_checkpoint(model, EncoderDecoder_checkpoint)
    print('=======================================')
    print('-    load EncoderDecoder complete     -')
    print('=======================================')
    # make submission.csv
    get_submission(model)        

-       load encoder complete         -
-       load decoder complete         -
-    load EncoderDecoder complete     -
NLP_CHEST_002
rouge1: Score(precision=0.275, recall=0.2619047619047619, fmeasure=0.26829268292682923)
rouge2: Score(precision=0.05128205128205128, recall=0.04878048780487805, fmeasure=0.05)
rougeL: Score(precision=0.25, recall=0.23809523809523808, fmeasure=0.24390243902439024)
NLP_CHEST_004
rouge1: Score(precision=0.375, recall=0.3333333333333333, fmeasure=0.35294117647058826)
rouge2: Score(precision=0.15384615384615385, recall=0.13636363636363635, fmeasure=0.14457831325301204)
rougeL: Score(precision=0.3, recall=0.26666666666666666, fmeasure=0.2823529411764706)
NLP_CHEST_011
rouge1: Score(precision=0.2391304347826087, recall=0.2619047619047619, fmeasure=0.25)
rouge2: Score(precision=0.06666666666666667, recall=0.07317073170731707, fmeasure=0.06976744186046512)
rougeL: Score(precision=0.15217391304347827, recall=0.16666666666666666, fmeasure=0.15909090909090912)
NLP_C